# Otimização Unificada - Sprint e Endurance

Este notebook unifica todos os casos de otimização:

## Casos de Otimização

### Caso 0: Baseline (Configuração Atual)
- Avalia a performance da configuração atual
- Sprint: esc_d = 1.0 (100% throttle)
- Endurance: encontra esc_d ótimo

### Caso 1: Otimização de trans_k (Hélice Fixo)
- **Sprint**: Otimiza trans_k com esc_d=1.0 fixo
- **Endurance**: Otimiza trans_k e esc_d

### Caso 2: Otimização Multi-Objetivo com Dois Hélices
- Um único trans_k para dois hélices diferentes (sprint e endurance)
- Formulação multi-objetivo usando NSGA-II
- Gera fronteira de Pareto para análise de trade-offs

In [1]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import control as ct
from typing import Optional

import sys

sys.path.append("../../../")
sys.path.append("../propeller")

from utils.plot import config_matplotlib, figsize, fig_save_and_show
from utils.optimization import load_model_params_from_json
from model import SolarBoatFull, SolarBoat

# Propeller helper functions
from helper import (
    _estimate_bseries_poly_coeffs,
    _estimate_prop_chord_07,
    _estimate_prop_I_r,
)

# pymoo imports
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.lhs import LHS
from pymoo.operators.mutation.pm import PM
from pymoo.operators.crossover.sbx import SBX
from pymoo.optimize import minimize
from pymoo.termination.default import (
    DefaultSingleObjectiveTermination,
    DefaultMultiObjectiveTermination,
)
from pymoo.parallelization.starmap import StarmapParallelization
from multiprocessing import Pool

import warnings

warnings.filterwarnings("ignore")

config_matplotlib()

## Constantes de Conversão

In [2]:
# Conversões de unidades
M_TO_INCH = 39.3701  # metros para polegadas
MPS_TO_KMH = 3.6  # m/s para km/h

## Parâmetros Globais

In [3]:
# Carregar parâmetros do modelo
solar_boat_params = load_model_params_from_json("solar_boat_params.json")

# === PARÂMETROS DA PROVA DE SPRINT ===
SPRINT_DISTANCE = 240.0  # metros
PV_G_SPRINT = 600.0  # Irradiância típica (W/m²)
SPRINT_MAX_TIME = 300.0  # Tempo máximo de simulação (s)
SPRINT_DT_OPT = 1  # SPRINT_MAX_TIME / 10.0  # dt para otimização (rápido)
SPRINT_DT_VAL = 1  # dt para validação (preciso)
SPRINT_ESC_D = 1.0  # Duty cycle fixo para sprint (100%)

# === PARÂMETROS DA PROVA DE ENDURANCE ===
ENDURANCE_DURATION = 5.0 * 3600.0  # 5 horas em segundos
ENDURANCE_DT_OPT = 1  # ENDURANCE_DURATION / 10.0  # dt para otimização
ENDURANCE_DT_VAL = 1.0  # dt para validação
PV_G_SCENARIOS = [200.0, 400.0, 600.0, 800.0]  # W/m²
PV_G_WEIGHTS = np.array([0.15, 0.35, 0.35, 0.15], dtype=float)
PV_G_WEIGHTS = PV_G_WEIGHTS / PV_G_WEIGHTS.sum()  # Normalizar

# === PARÂMETROS COMUNS ===
BATT_Z_INITIAL = 1.0  # Estado de carga inicial (100%)
BATT_Z_MIN = 0.1  # Estado de carga mínimo (10%)
BATT_V_INITIAL = 36.0  # Tensão inicial da bateria (V)
MOTOR_Q_MAX = 30.0  # Torque máximo do motor (Nm)
TRANSIENT_SKIP = 10  # Número de timesteps a ignorar no transitório

# === LIMITES DE OTIMIZAÇÃO ===
TRANS_K_MIN, TRANS_K_MAX = 0.1, 10
ESC_D_END_MIN, ESC_D_END_MAX = 0.1, 0.8

# === PARÂMETROS DO HÉLICE ===
PROP_D_MIN, PROP_D_MAX = 0.15, 0.30  # Diâmetro (m)
PROP_PD_MIN, PROP_PD_MAX = 0.6, 1.6  # P/D ratio
PROP_Z = 3  # Número de pás (fixo)
PROP_AEA0 = 0.242  # Razão de área expandida (fixo)
PROP_HUB_D = 0.06  # Diâmetro do hub (m)
PROP_BLADE_THICKNESS = 0.005  # Espessura da pá (m)
PROP_MAT_RHO = 2700.0  # Densidade do material (kg/m³)
PROP_TOTAL_M = 0.66  # Massa total do hélice (kg)

# === CONFIGURAÇÃO ATUAL ===
CURRENT_TRANS_K = solar_boat_params["trans_k"]
CURRENT_PROP_D = solar_boat_params["prop_D"]
CURRENT_PROP_PD = 10.5 / 9  # Pitch 10.5" / Diameter 9"
CURRENT_PROP_P = CURRENT_PROP_PD * CURRENT_PROP_D

print(
    f'Hélice atual: D={CURRENT_PROP_D * M_TO_INCH:.2f}" ({CURRENT_PROP_D * 100:.1f}cm)'
)
print(
    f'              P/D={CURRENT_PROP_PD:.3f}, P={CURRENT_PROP_P * M_TO_INCH:.2f}" ({CURRENT_PROP_P * 100:.1f}cm)'
)
print(f"trans_k atual: {CURRENT_TRANS_K:.4f}")
print(f"Torque máximo motor: {MOTOR_Q_MAX} Nm")
print(f"Sprint esc_d fixo: {SPRINT_ESC_D * 100:.0f}%")

Hélice atual: D=9.00" (22.9cm)
              P/D=1.167, P=10.50" (26.7cm)
trans_k atual: 0.6364
Torque máximo motor: 30.0 Nm
Sprint esc_d fixo: 100%


## Funções Auxiliares

In [4]:
def create_propeller_params(prop_D: float, prop_PD: float, Rn: float = 2e6) -> dict:
    """Cria os parâmetros do hélice usando as funções B-series."""
    prop_k_T_coeffs, prop_k_Q_coeffs = _estimate_bseries_poly_coeffs(
        prop_PD=prop_PD,
        prop_AEA0=PROP_AEA0,
        prop_Z=PROP_Z,
        Rn=Rn,
    )

    geom = _estimate_prop_chord_07(
        prop_D=prop_D,
        prop_hub_D=PROP_HUB_D,
        prop_AEA0=PROP_AEA0,
        prop_Z=PROP_Z,
    )

    inertia = _estimate_prop_I_r(
        prop_blade_thickness=PROP_BLADE_THICKNESS,
        prop_mat_rho=PROP_MAT_RHO,
        prop_AE=geom["prop_AE"],
        prop_Z=PROP_Z,
        prop_hub_R=geom["prop_hub_R"],
        prop_L_radial=geom["prop_L_radial"],
        prop_total_m=PROP_TOTAL_M,
    )
    prop_I_r = inertia["J_total"] if inertia is not None else 0.0006

    return {
        "prop_D": prop_D,
        "prop_PD": prop_PD,
        "prop_P": prop_PD * prop_D,
        "prop_k_T_coeffs": prop_k_T_coeffs,
        "prop_k_Q_coeffs": prop_k_Q_coeffs,
        "prop_I_r": prop_I_r,
    }


def _build_params_with_propeller(
    base_params: dict, trans_k: float, prop_params: dict
) -> dict:
    """Constrói parâmetros de simulação com hélice customizado."""
    p = base_params.copy()
    p["trans_k"] = float(trans_k)
    p["prop_D"] = float(prop_params["prop_D"])
    p["prop_k_T_coeffs"] = prop_params["prop_k_T_coeffs"]
    p["prop_k_Q_coeffs"] = prop_params["prop_k_Q_coeffs"]
    p["prop_I_r"] = float(prop_params["prop_I_r"])
    return p

## Funções de Simulação

In [5]:
def simulate_sprint(
    params: dict, trans_k: float, prop_params: dict, esc_d: float, dt: float
) -> dict:
    """Simula a prova de sprint.

    Retorna métricas excluindo o transitório inicial (TRANSIENT_SKIP timesteps).
    """
    sim_params = _build_params_with_propeller(
        params, trans_k=trans_k, prop_params=prop_params
    )
    model = SolarBoatFull
    sys = model.build(sim_params)

    T = np.arange(0.0, SPRINT_MAX_TIME + dt, dt)
    U = np.array([[PV_G_SPRINT] * len(T), [float(esc_d)] * len(T)])
    X0 = model.initial_state(
        X0={"batt_z": BATT_Z_INITIAL, "batt_v": BATT_V_INITIAL},
        U0=U[:, 0],
        params=sim_params,
    )

    try:
        res = ct.input_output_response(sys, T=T, U=U, X0=X0, solve_ivp_method="Radau")
        df = res.to_pandas()
        df["distance"] = np.cumsum(df["hull_u"].values * dt)

        idx_complete = np.where(df["distance"].values >= SPRINT_DISTANCE)[0]
        if len(idx_complete) > 0:
            idx = int(idx_complete[0])
            time_to_complete = float(df["time"].iloc[idx])
            batt_z_final = float(df["batt_z"].iloc[idx])
            batt_z_min_during = float(df["batt_z"].iloc[: idx + 1].min())

            # Torque máximo excluindo transitório
            start_idx = min(TRANSIENT_SKIP, idx)
            motor_q_max_during = (
                float(df["motor_q_load"].iloc[start_idx : idx + 1].max())
                if "motor_q_load" in df.columns
                else 0.0
            )

            avg_speed = SPRINT_DISTANCE / max(time_to_complete, 1e-9)
            distance = SPRINT_DISTANCE

            # Calcular esc_d médio (deve ser constante, mas calculamos para consistência)
            esc_d_mean = float(esc_d)

            # Velocidade média em km/h
            hull_u_kmh = avg_speed * MPS_TO_KMH

            # Métricas adicionais (médias excluindo transitório)
            df_valid = df.iloc[start_idx : idx + 1]
            motor_w_mean = (
                float(df_valid["motor_w"].mean()) if "motor_w" in df.columns else 0.0
            )
            prop_j_mean = (
                float(df_valid["prop_j"].mean()) if "prop_j" in df.columns else 0.0
            )
            prop_w_mean = (
                float(df_valid["prop_w"].mean()) if "prop_w" in df.columns else 0.0
            )

            return {
                "success": batt_z_min_during >= BATT_Z_MIN,
                "time": time_to_complete,
                "avg_speed": float(avg_speed),
                "hull_u_kmh": hull_u_kmh,
                "distance": distance,
                "distance_km": distance / 1000,
                "batt_z_final": batt_z_final,
                "batt_z_min_during": batt_z_min_during,
                "motor_q_max": motor_q_max_during,
                "esc_d_mean": esc_d_mean,
                "motor_w_mean": motor_w_mean,
                "prop_j_mean": prop_j_mean,
                "prop_w_mean": prop_w_mean,
                "df": df.iloc[: idx + 1],
            }

        # Não completou a prova
        batt_z_min_during = float(df["batt_z"].min())
        start_idx = min(TRANSIENT_SKIP, len(df) - 1)
        motor_q_max_during = (
            float(df["motor_q_load"].iloc[start_idx:].max())
            if "motor_q_load" in df.columns
            else 0.0
        )
        distance = float(df["distance"].iloc[-1])

        # Métricas adicionais
        df_valid = df.iloc[start_idx:]
        motor_w_mean = (
            float(df_valid["motor_w"].mean()) if "motor_w" in df.columns else 0.0
        )
        prop_j_mean = (
            float(df_valid["prop_j"].mean()) if "prop_j" in df.columns else 0.0
        )
        prop_w_mean = (
            float(df_valid["prop_w"].mean()) if "prop_w" in df.columns else 0.0
        )

        return {
            "success": False,
            "time": float(SPRINT_MAX_TIME),
            "avg_speed": float(distance / max(SPRINT_MAX_TIME, 1e-9)),
            "hull_u_kmh": float(distance / max(SPRINT_MAX_TIME, 1e-9)) * MPS_TO_KMH,
            "distance": distance,
            "distance_km": distance / 1000,
            "batt_z_final": float(df["batt_z"].iloc[-1]),
            "batt_z_min_during": batt_z_min_during,
            "motor_q_max": motor_q_max_during,
            "esc_d_mean": float(esc_d),
            "motor_w_mean": motor_w_mean,
            "prop_j_mean": prop_j_mean,
            "prop_w_mean": prop_w_mean,
            "df": df,
        }
    except Exception as e:
        return {
            "success": False,
            "time": float(SPRINT_MAX_TIME),
            "avg_speed": 0.0,
            "hull_u_kmh": 0.0,
            "distance": 0.0,
            "distance_km": 0.0,
            "batt_z_final": 0.0,
            "batt_z_min_during": 0.0,
            "motor_q_max": 999.0,
            "esc_d_mean": float(esc_d),
            "motor_w_mean": 0.0,
            "prop_j_mean": 0.0,
            "prop_w_mean": 0.0,
            "df": None,
            "error": str(e),
        }


def simulate_endurance(
    params: dict,
    trans_k: float,
    prop_params: dict,
    esc_d: float,
    pv_g: float,
    dt: float,
) -> dict:
    """Simula a prova de endurance.

    Retorna métricas excluindo o transitório inicial (TRANSIENT_SKIP timesteps).
    """
    sim_params = _build_params_with_propeller(
        params, trans_k=trans_k, prop_params=prop_params
    )
    model = SolarBoatFull
    sys = model.build(sim_params)

    T = np.arange(0.0, ENDURANCE_DURATION + dt, dt)
    U = np.array([[float(pv_g)] * len(T), [float(esc_d)] * len(T)])
    X0 = model.initial_state(
        X0={"batt_z": BATT_Z_INITIAL, "batt_v": BATT_V_INITIAL},
        U0=U[:, 0],
        params=sim_params,
    )

    try:
        res = ct.input_output_response(sys, T=T, U=U, X0=X0, solve_ivp_method="Radau")
        df = res.to_pandas()
        df["distance"] = np.cumsum(df["hull_u"].values * dt)
        batt_z_min_during = float(df["batt_z"].min())

        # Torque máximo excluindo transitório
        start_idx = min(TRANSIENT_SKIP, len(df) - 1)
        motor_q_max_during = (
            float(df["motor_q_load"].iloc[start_idx:].max())
            if "motor_q_load" in df.columns
            else 0.0
        )

        distance = float(df["distance"].iloc[-1])
        avg_speed = float(distance / max(ENDURANCE_DURATION, 1e-9))

        # Métricas adicionais (médias excluindo transitório)
        df_valid = df.iloc[start_idx:]
        motor_w_mean = (
            float(df_valid["motor_w"].mean()) if "motor_w" in df.columns else 0.0
        )
        prop_j_mean = (
            float(df_valid["prop_j"].mean()) if "prop_j" in df.columns else 0.0
        )
        prop_w_mean = (
            float(df_valid["prop_w"].mean()) if "prop_w" in df.columns else 0.0
        )

        return {
            "distance": distance,
            "distance_km": distance / 1000,
            "avg_speed": avg_speed,
            "hull_u_kmh": avg_speed * MPS_TO_KMH,
            "batt_z_final": float(df["batt_z"].iloc[-1]),
            "batt_z_min_during": batt_z_min_during,
            "motor_q_max": motor_q_max_during,
            "esc_d_mean": float(esc_d),
            "motor_w_mean": motor_w_mean,
            "prop_j_mean": prop_j_mean,
            "prop_w_mean": prop_w_mean,
            "success": batt_z_min_during >= BATT_Z_MIN,
            "df": df,
        }
    except Exception as e:
        return {
            "distance": 0.0,
            "distance_km": 0.0,
            "avg_speed": 0.0,
            "hull_u_kmh": 0.0,
            "batt_z_final": 0.0,
            "batt_z_min_during": 0.0,
            "motor_q_max": 999.0,
            "esc_d_mean": float(esc_d),
            "motor_w_mean": 0.0,
            "prop_j_mean": 0.0,
            "prop_w_mean": 0.0,
            "success": False,
            "df": None,
            "error": str(e),
        }

## Caso 0: Baseline (Configuração Atual)

Avalia a performance da configuração atual:
- Sprint: esc_d = 1.0 (100% throttle, fixo)
- Endurance: encontra esc_d ótimo

In [6]:
# Parâmetros do hélice atual
current_prop_params = {
    "prop_D": CURRENT_PROP_D,
    "prop_PD": CURRENT_PROP_PD,
    "prop_P": CURRENT_PROP_P,
    "prop_k_T_coeffs": solar_boat_params["prop_k_T_coeffs"],
    "prop_k_Q_coeffs": solar_boat_params["prop_k_Q_coeffs"],
    "prop_I_r": solar_boat_params["prop_I_r"],
}

print("=" * 80)
print("CASO 0: BASELINE - Configuração Atual")
print("=" * 80)

CASO 0: BASELINE - Configuração Atual


In [7]:
# Baseline Sprint (esc_d = 1.0 fixo)
print("\nSimulando Baseline Sprint (esc_d = 100%)...")
res_baseline_sprint = simulate_sprint(
    params=solar_boat_params,
    trans_k=CURRENT_TRANS_K,
    prop_params=current_prop_params,
    esc_d=SPRINT_ESC_D,
    dt=SPRINT_DT_VAL,
)

BASELINE_SPRINT_SPEED = res_baseline_sprint["avg_speed"]
print(
    f"  Velocidade: {BASELINE_SPRINT_SPEED:.2f} m/s ({res_baseline_sprint['hull_u_kmh']:.2f} km/h)"
)
print(f"  Tempo: {res_baseline_sprint['time']:.2f} s")
print(f"  Torque max: {res_baseline_sprint['motor_q_max']:.2f} Nm")


Simulando Baseline Sprint (esc_d = 100%)...
  Velocidade: 5.00 m/s (18.00 km/h)
  Tempo: 48.00 s
  Torque max: 15.32 Nm


In [8]:
# Problema para encontrar esc_d ótimo no Endurance (baseline)
class BaselineEnduranceProblem(ElementwiseProblem):
    def __init__(self, base_params: dict, prop_params: dict, trans_k: float, **kwargs):
        self.base_params = base_params.copy()
        self.prop_params = prop_params
        self.trans_k = trans_k
        n_scen = len(PV_G_SCENARIOS)
        super().__init__(
            n_var=1,
            n_obj=1,
            n_ieq_constr=2 * n_scen,
            xl=np.array([ESC_D_END_MIN]),
            xu=np.array([ESC_D_END_MAX]),
            **kwargs,
        )

    def _evaluate(self, x, out, *args, **kwargs):
        esc_d = float(x[0])
        distances = []
        batt_constraints = []
        torque_constraints = []

        for pv_g in PV_G_SCENARIOS:
            res = simulate_endurance(
                params=self.base_params,
                trans_k=self.trans_k,
                prop_params=self.prop_params,
                esc_d=esc_d,
                pv_g=pv_g,
                dt=ENDURANCE_DT_OPT,
            )
            distances.append(res["distance"])
            batt_constraints.append(BATT_Z_MIN - res["batt_z_min_during"])
            torque_constraints.append(res["motor_q_max"] - MOTOR_Q_MAX)

        weighted_distance = float(np.sum(np.array(distances) * PV_G_WEIGHTS))
        out["F"] = -weighted_distance
        out["G"] = np.array(batt_constraints + torque_constraints)

In [9]:
%%time
# Otimizar esc_d para Endurance (baseline)
print("\nOtimizando esc_d para Baseline Endurance...")
n_processes = max(16, os.cpu_count() or 1)
pool = Pool(processes=n_processes)
runner = StarmapParallelization(pool.starmap)

try:
    problem_baseline_endurance = BaselineEnduranceProblem(
        base_params=solar_boat_params,
        prop_params=current_prop_params,
        trans_k=CURRENT_TRANS_K,
        elementwise_runner=runner,
    )

    # Usar DE com mais exploração para problema não-convexo
    algorithm = DE(
        pop_size=100,
        sampling=LHS(),
        variant="DE/rand/1/bin",
        # variant="DE/best/1/bin",
        F=0.5,
        CR=0.9,
        dither='vector',
    )
    termination = DefaultSingleObjectiveTermination(
        xtol=1e-8, cvtol=1e-8, ftol=1e-8, period=20, n_max_gen=1000
    )

    result_baseline_endurance = minimize(
        problem_baseline_endurance, algorithm, termination, seed=42, verbose=False
    )

finally:
    pool.close()
    pool.join()

BASELINE_ESC_D_ENDURANCE = float(result_baseline_endurance.X[0])
print(
    f"Baseline Endurance: esc_d ótimo = {BASELINE_ESC_D_ENDURANCE:.4f} ({BASELINE_ESC_D_ENDURANCE * 100:.1f}%)"
)


Otimizando esc_d para Baseline Endurance...
Baseline Endurance: esc_d ótimo = 0.4001 (40.0%)
CPU times: user 2.18 s, sys: 461 ms, total: 2.64 s
Wall time: 9min 26s


In [10]:
# Validar baseline endurance com dt fino
print("\nValidando Baseline Endurance...")
baseline_endurance_results = []
for pv_g in PV_G_SCENARIOS:
    res = simulate_endurance(
        params=solar_boat_params,
        trans_k=CURRENT_TRANS_K,
        prop_params=current_prop_params,
        esc_d=BASELINE_ESC_D_ENDURANCE,
        pv_g=pv_g,
        dt=ENDURANCE_DT_VAL,
    )
    baseline_endurance_results.append(res)

BASELINE_ENDURANCE_DISTANCE = float(
    np.sum(np.array([r["distance"] for r in baseline_endurance_results]) * PV_G_WEIGHTS)
)
BASELINE_ENDURANCE_SPEED = BASELINE_ENDURANCE_DISTANCE / ENDURANCE_DURATION

print(f"  Distância (média ponderada): {BASELINE_ENDURANCE_DISTANCE / 1000:.2f} km")
print(
    f"  Velocidade: {BASELINE_ENDURANCE_SPEED:.2f} m/s ({BASELINE_ENDURANCE_SPEED * MPS_TO_KMH:.2f} km/h)"
)


Validando Baseline Endurance...
  Distância (média ponderada): 42.44 km
  Velocidade: 2.36 m/s (8.49 km/h)


## Caso 1: Otimização de trans_k (Hélice Fixo)

In [11]:
print("\n" + "=" * 80)
print("CASO 1: Otimização de trans_k (Hélice Fixo)")
print("=" * 80)


CASO 1: Otimização de trans_k (Hélice Fixo)


In [12]:
# Problema Caso 1 - Sprint (apenas trans_k, esc_d=1.0 fixo)
class Case1SprintProblem(ElementwiseProblem):
    def __init__(self, base_params: dict, prop_params: dict, **kwargs):
        self.base_params = base_params.copy()
        self.prop_params = prop_params
        super().__init__(
            n_var=1,  # Apenas trans_k
            n_obj=1,
            n_ieq_constr=2,
            xl=np.array([TRANS_K_MIN]),
            xu=np.array([TRANS_K_MAX]),
            **kwargs,
        )

    def _evaluate(self, x, out, *args, **kwargs):
        trans_k = float(x[0])
        res = simulate_sprint(
            params=self.base_params,
            trans_k=trans_k,
            prop_params=self.prop_params,
            esc_d=SPRINT_ESC_D,  # Fixo em 1.0
            dt=SPRINT_DT_OPT,
        )
        out["F"] = res["time"]
        out["G"] = np.array(
            [
                BATT_Z_MIN - res["batt_z_min_during"],
                res["motor_q_max"] - MOTOR_Q_MAX,
            ]
        )

In [13]:
# Problema Caso 1 - Endurance (trans_k e esc_d)
class Case1EnduranceProblem(ElementwiseProblem):
    def __init__(self, base_params: dict, prop_params: dict, **kwargs):
        self.base_params = base_params.copy()
        self.prop_params = prop_params
        n_scen = len(PV_G_SCENARIOS)
        super().__init__(
            n_var=2,
            n_obj=1,
            n_ieq_constr=2 * n_scen,
            xl=np.array([TRANS_K_MIN, ESC_D_END_MIN]),
            xu=np.array([TRANS_K_MAX, ESC_D_END_MAX]),
            **kwargs,
        )

    def _evaluate(self, x, out, *args, **kwargs):
        trans_k = float(x[0])
        esc_d = float(x[1])
        distances = []
        batt_constraints = []
        torque_constraints = []

        for pv_g in PV_G_SCENARIOS:
            res = simulate_endurance(
                params=self.base_params,
                trans_k=trans_k,
                prop_params=self.prop_params,
                esc_d=esc_d,
                pv_g=pv_g,
                dt=ENDURANCE_DT_OPT,
            )
            distances.append(res["distance"])
            batt_constraints.append(BATT_Z_MIN - res["batt_z_min_during"])
            torque_constraints.append(res["motor_q_max"] - MOTOR_Q_MAX)

        weighted_distance = float(np.sum(np.array(distances) * PV_G_WEIGHTS))
        out["F"] = -weighted_distance
        out["G"] = np.array(batt_constraints + torque_constraints)

In [14]:
%%time
# Caso 1 - Sprint
print("\nOtimizando Caso 1 - Sprint (trans_k, esc_d=100% fixo)...")
pool = Pool(processes=n_processes)
runner = StarmapParallelization(pool.starmap)

try:
    problem_case1_sprint = Case1SprintProblem(
        base_params=solar_boat_params,
        prop_params=current_prop_params,
        elementwise_runner=runner,
    )

    # DE com mais exploração
    algorithm = DE(
        pop_size=100,
        sampling=LHS(),
        variant="DE/rand/1/bin",
        # variant="DE/best/1/bin",
        F=0.5,
        CR=0.9,
        dither='vector',
    )
    termination = DefaultSingleObjectiveTermination(
        xtol=1e-8, cvtol=1e-8, ftol=1e-8, period=20, n_max_gen=1000
    )

    result_case1_sprint = minimize(
        problem_case1_sprint, algorithm, termination, seed=42, verbose=True
    )

finally:
    pool.close()
    pool.join()

CASE1_TRANS_K_SPRINT = float(result_case1_sprint.X[0])
CASE1_ESC_D_SPRINT = SPRINT_ESC_D  # Fixo
print(
    f"\nCaso 1 Sprint: trans_k={CASE1_TRANS_K_SPRINT:.4f}, esc_d={CASE1_ESC_D_SPRINT * 100:.0f}%"
)


Otimizando Caso 1 - Sprint (trans_k, esc_d=100% fixo)...
n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      100 |  0.000000E+00 |  3.023334E+01 |  7.187500E+01 |  4.300000E+01
     2 |      200 |  0.000000E+00 |  2.432935E+01 |  8.861111E+01 |  4.300000E+01
     3 |      300 |  0.000000E+00 |  1.586246E+01 |  9.543243E+01 |  4.300000E+01
     4 |      400 |  0.000000E+00 |  8.6119253977 |  8.532727E+01 |  4.300000E+01
     5 |      500 |  0.000000E+00 |  3.8676851195 |  7.288000E+01 |  4.200000E+01
     6 |      600 |  0.000000E+00 |  0.2735070186 |  6.898947E+01 |  4.200000E+01
     7 |      700 |  0.000000E+00 |  0.000000E+00 |  5.989000E+01 |  4.200000E+01
     8 |      800 |  0.000000E+00 |  0.000000E+00 |  5.010000E+01 |  4.200000E+01
     9 |      900 |  0.000000E+00 |  0.000000E+00 |  4.481000E+01 |  4.200000E+01
    10 |     1000 |  0.000000E+00 |  0.000000E+00 |  4.358000E+01 |  4.200000E+01
    11 |     1100 |  0.000000E+00 |  0.0

In [15]:
%%time
# Caso 1 - Endurance
print("\nOtimizando Caso 1 - Endurance (trans_k e esc_d)...")
pool = Pool(processes=n_processes)
runner = StarmapParallelization(pool.starmap)

try:
    problem_case1_endurance = Case1EnduranceProblem(
        base_params=solar_boat_params,
        prop_params=current_prop_params,
        elementwise_runner=runner,
    )

    algorithm = DE(
        pop_size=100,
        sampling=LHS(),
        variant="DE/rand/1/bin",
        # variant="DE/best/1/bin",
        F=0.5,
        CR=0.9,
        dither='vector',
    )
    termination = DefaultSingleObjectiveTermination(
        xtol=1e-8, cvtol=1e-8, ftol=1e-8, period=20, n_max_gen=1000
    )

    result_case1_endurance = minimize(
        problem_case1_endurance, algorithm, termination, seed=42, verbose=True
    )

finally:
    pool.close()
    pool.join()

CASE1_TRANS_K_ENDURANCE = float(result_case1_endurance.X[0])
CASE1_ESC_D_ENDURANCE = float(result_case1_endurance.X[1])
print(
    f"\nCaso 1 Endurance: trans_k={CASE1_TRANS_K_ENDURANCE:.4f}, esc_d={CASE1_ESC_D_ENDURANCE * 100:.1f}%"
)


Otimizando Caso 1 - Endurance (trans_k e esc_d)...
n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |      100 |  0.000000E+00 |  7.552183E+01 | -2.654168E+04 | -3.989399E+04
     2 |      200 |  0.000000E+00 |  4.442505E+01 | -2.563085E+04 | -3.989399E+04
     3 |      300 |  0.000000E+00 |  1.901783E+01 | -2.392140E+04 | -3.989399E+04
     4 |      400 |  0.000000E+00 |  5.4828301949 | -2.543385E+04 | -4.222878E+04
     5 |      500 |  0.000000E+00 |  1.1817010945 | -2.537728E+04 | -4.222878E+04
     6 |      600 |  0.000000E+00 |  0.2603719004 | -2.556481E+04 | -4.222878E+04
     7 |      700 |  0.000000E+00 |  0.0568642670 | -2.764259E+04 | -4.222878E+04
     8 |      800 |  0.000000E+00 |  0.0043791005 | -3.016446E+04 | -4.247618E+04
     9 |      900 |  0.000000E+00 |  0.0043791005 | -3.243275E+04 | -4.247618E+04
    10 |     1000 |  0.000000E+00 |  0.000000E+00 | -3.416844E+04 | -4.247618E+04
    11 |     1100 |  0.000000E+00 |  0.000000E

In [16]:
# Validar Caso 1
print("\nValidando Caso 1...")

res_case1_sprint = simulate_sprint(
    params=solar_boat_params,
    trans_k=CASE1_TRANS_K_SPRINT,
    prop_params=current_prop_params,
    esc_d=CASE1_ESC_D_SPRINT,
    dt=SPRINT_DT_VAL,
)

case1_endurance_results = []
for pv_g in PV_G_SCENARIOS:
    res = simulate_endurance(
        params=solar_boat_params,
        trans_k=CASE1_TRANS_K_ENDURANCE,
        prop_params=current_prop_params,
        esc_d=CASE1_ESC_D_ENDURANCE,
        pv_g=pv_g,
        dt=ENDURANCE_DT_VAL,
    )
    case1_endurance_results.append(res)

CASE1_SPRINT_SPEED = res_case1_sprint["avg_speed"]
CASE1_ENDURANCE_DISTANCE = float(
    np.sum(np.array([r["distance"] for r in case1_endurance_results]) * PV_G_WEIGHTS)
)

print(f"\n--- Caso 1 Sprint ---")
print(f"  trans_k: {CASE1_TRANS_K_SPRINT:.4f}")
print(f"  esc_d: {CASE1_ESC_D_SPRINT * 100:.0f}%")
print(
    f"  Velocidade: {CASE1_SPRINT_SPEED:.2f} m/s ({res_case1_sprint['hull_u_kmh']:.2f} km/h)"
)
print(f"  vs Baseline: {(CASE1_SPRINT_SPEED / BASELINE_SPRINT_SPEED - 1) * 100:+.1f}%")

print(f"\n--- Caso 1 Endurance ---")
print(f"  trans_k: {CASE1_TRANS_K_ENDURANCE:.4f}")
print(f"  esc_d: {CASE1_ESC_D_ENDURANCE * 100:.1f}%")
print(f"  Distância: {CASE1_ENDURANCE_DISTANCE / 1000:.2f} km")
print(
    f"  vs Baseline: {(CASE1_ENDURANCE_DISTANCE / BASELINE_ENDURANCE_DISTANCE - 1) * 100:+.1f}%"
)


Validando Caso 1...

--- Caso 1 Sprint ---
  trans_k: 0.9250
  esc_d: 100%
  Velocidade: 5.71 m/s (20.57 km/h)
  vs Baseline: +14.3%

--- Caso 1 Endurance ---
  trans_k: 0.4773
  esc_d: 52.8%
  Distância: 42.78 km
  vs Baseline: +0.8%


## Caso 2: Otimização Multi-Objetivo com Dois Hélices (NSGA-II)

In [17]:
print("\n" + "=" * 80)
print("CASO 2: Otimização Multi-Objetivo com Dois Hélices (NSGA-II)")
print("=" * 80)


CASO 2: Otimização Multi-Objetivo com Dois Hélices (NSGA-II)


In [18]:
class Case2MultiObjectiveProblem(ElementwiseProblem):
    """
    Variáveis de decisão (5):
        0: trans_k
        1: esc_d_endurance
        2: prop_D_sprint [m]
        3: prop_PD_sprint [-]
        4: prop_D_endurance [m]
        5: prop_PD_endurance [-]

    Nota: esc_d_sprint = 1.0 (fixo)

    Objetivos (2):
        - f1: Minimizar -velocidade_sprint (maximizar velocidade)
        - f2: Minimizar -distância_endurance (maximizar distância)
    """

    def __init__(self, base_params: dict, **kwargs):
        self.base_params = base_params.copy()
        n_scen = len(PV_G_SCENARIOS)
        n_constr = 2 + 2 * n_scen
        super().__init__(
            n_var=6,
            n_obj=2,
            n_ieq_constr=n_constr,
            xl=np.array(
                [
                    TRANS_K_MIN,
                    ESC_D_END_MIN,
                    PROP_D_MIN,
                    PROP_PD_MIN,
                    PROP_D_MIN,
                    PROP_PD_MIN,
                ]
            ),
            xu=np.array(
                [
                    TRANS_K_MAX,
                    ESC_D_END_MAX,
                    PROP_D_MAX,
                    PROP_PD_MAX,
                    PROP_D_MAX,
                    PROP_PD_MAX,
                ]
            ),
            **kwargs,
        )

    def _evaluate(self, x, out, *args, **kwargs):
        trans_k = float(x[0])
        esc_d_end = float(x[1])
        prop_D_sprint = float(x[2])
        prop_PD_sprint = float(x[3])
        prop_D_end = float(x[4])
        prop_PD_end = float(x[5])

        prop_sprint = create_propeller_params(prop_D_sprint, prop_PD_sprint)
        prop_end = create_propeller_params(prop_D_end, prop_PD_end)

        # Sprint (esc_d = 1.0 fixo)
        res_s = simulate_sprint(
            params=self.base_params,
            trans_k=trans_k,
            prop_params=prop_sprint,
            esc_d=SPRINT_ESC_D,
            dt=SPRINT_DT_OPT,
        )

        # Endurance
        distances = []
        end_batt_constraints = []
        end_torque_constraints = []
        for pv_g in PV_G_SCENARIOS:
            res_e = simulate_endurance(
                params=self.base_params,
                trans_k=trans_k,
                prop_params=prop_end,
                esc_d=esc_d_end,
                pv_g=pv_g,
                dt=ENDURANCE_DT_OPT,
            )
            distances.append(res_e["distance"])
            end_batt_constraints.append(BATT_Z_MIN - res_e["batt_z_min_during"])
            end_torque_constraints.append(res_e["motor_q_max"] - MOTOR_Q_MAX)

        weighted_distance = float(np.sum(np.array(distances) * PV_G_WEIGHTS))

        out["F"] = np.array(
            [
                -res_s["avg_speed"],
                -weighted_distance,
            ]
        )

        g_sprint_batt = BATT_Z_MIN - res_s["batt_z_min_during"]
        g_sprint_torque = res_s["motor_q_max"] - MOTOR_Q_MAX
        out["G"] = np.array(
            [g_sprint_batt, g_sprint_torque]
            + end_batt_constraints
            + end_torque_constraints,
            dtype=float,
        )

In [19]:
%%time
# Caso 2 - Multi-objetivo
print("\nOtimizando Caso 2 - Multi-objetivo (NSGA-II)...")
pool = Pool(processes=n_processes)
runner = StarmapParallelization(pool.starmap)

try:
    problem_case2 = Case2MultiObjectiveProblem(
        base_params=solar_boat_params,
        elementwise_runner=runner,
    )

    algorithm = NSGA2(
        pop_size=100,
        sampling=LHS(),
        crossover=SBX(prob=0.9, eta=15),
        mutation=PM(eta=20),
    )

    termination = DefaultMultiObjectiveTermination(
        xtol=1e-8,
        cvtol=1e-8,
        ftol=1e-6,
        period=30,
        n_max_gen=1000,
    )

    result_case2 = minimize(
        problem_case2,
        algorithm,
        termination,
        seed=42,
        verbose=True,
        save_history=True,
    )

finally:
    pool.close()
    pool.join()

print(f"\nNúmero de soluções na fronteira de Pareto: {len(result_case2.X)}")


Otimizando Caso 2 - Multi-objetivo (NSGA-II)...
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      2 |  0.000000E+00 |  9.593517E+01 |             - |             -
     2 |      200 |      3 |  0.000000E+00 |  2.456552E+01 |  0.1771949822 |         nadir
     3 |      300 |      4 |  0.000000E+00 |  2.9409940023 |  0.4202020202 |         ideal
     4 |      400 |      3 |  0.000000E+00 |  0.000000E+00 |  0.2071221473 |         ideal
     5 |      500 |      6 |  0.000000E+00 |  0.000000E+00 |  0.1533695543 |             f
     6 |      600 |      4 |  0.000000E+00 |  0.000000E+00 |  2.0701183863 |         nadir
     7 |      700 |      2 |  0.000000E+00 |  0.000000E+00 |  0.8822949699 |         ideal
     8 |      800 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     9 |      900 |      3 |  0.000000E+00 |  0.000000E+00 |  0.2044315685 |         ideal
    10 |     1000 |      3 |  0.000000E+0

In [20]:
# Extrair e analisar fronteira de Pareto
pareto_X = result_case2.X
pareto_F = -result_case2.F

# Garantir que é 2D
if pareto_X.ndim == 1:
    pareto_X = pareto_X.reshape(1, -1)
    pareto_F = pareto_F.reshape(1, -1)

df_pareto = pd.DataFrame(
    {
        "trans_k": pareto_X[:, 0],
        "esc_d_sprint [%]": SPRINT_ESC_D * 100,
        "esc_d_endurance [%]": pareto_X[:, 1] * 100,
        "prop_D_sprint [in]": pareto_X[:, 2] * M_TO_INCH,
        "prop_PD_sprint": pareto_X[:, 3],
        "prop_P_sprint [in]": pareto_X[:, 2] * pareto_X[:, 3] * M_TO_INCH,
        "prop_D_endurance [in]": pareto_X[:, 4] * M_TO_INCH,
        "prop_PD_endurance": pareto_X[:, 5],
        "prop_P_endurance [in]": pareto_X[:, 4] * pareto_X[:, 5] * M_TO_INCH,
        "sprint_speed [m/s]": pareto_F[:, 0],
        "sprint_speed [km/h]": pareto_F[:, 0] * MPS_TO_KMH,
        "endurance_distance [km]": pareto_F[:, 1] / 1000,
    }
)

df_pareto = df_pareto.sort_values("sprint_speed [m/s]", ascending=False).reset_index(
    drop=True
)

print("\n--- Fronteira de Pareto ---")
display(df_pareto.round(2))


--- Fronteira de Pareto ---


,trans_k,esc_d_sprint [%],esc_d_endurance [%],prop_D_sprint [in],prop_PD_sprint,prop_P_sprint [in],prop_D_endurance [in],prop_PD_endurance,prop_P_endurance [in],sprint_speed [m/s],sprint_speed [km/h],endurance_distance [km]
0,0.51,100.0,43.97,11.81,1.60,18.89,10.90,0.91,9.90,6.00,21.60,43.93
1,0.51,100.0,43.97,11.81,1.60,18.87,10.90,0.91,9.90,6.00,21.60,43.93
2,0.51,100.0,43.97,11.81,1.60,18.88,10.90,0.91,9.90,6.00,21.60,43.93
3,0.48,100.0,43.97,11.66,1.60,18.62,11.22,0.92,10.37,5.85,21.07,44.06
4,0.48,100.0,43.97,11.68,1.58,18.41,11.22,0.92,10.37,5.85,21.07,44.06
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.48,100.0,43.97,11.70,1.60,18.70,11.22,0.92,10.37,5.85,21.07,44.06
96,0.48,100.0,43.97,11.80,1.54,18.12,11.22,0.92,10.37,5.85,21.07,44.06
97,0.48,100.0,43.97,11.61,1.58,18.39,11.22,0.92,10.37,5.85,21.07,44.06
98,0.48,100.0,43.97,11.70,1.56,18.29,11.22,0.92,10.37,5.85,21.07,44.06


## Tabelas de Resultados Consolidadas

In [21]:
print("\n" + "=" * 80)
print("RESULTADOS CONSOLIDADOS")
print("=" * 80)


RESULTADOS CONSOLIDADOS


In [22]:
# Tabela 1: Performance Sprint
print("\n--- Tabela 1: Performance Sprint ---")

df_sprint = pd.DataFrame(
    {
        "Caso": ["Baseline", "Caso 1"],
        "trans_k": [CURRENT_TRANS_K, CASE1_TRANS_K_SPRINT],
        "esc_d [%]": [SPRINT_ESC_D * 100, CASE1_ESC_D_SPRINT * 100],
        "prop_D [in]": [CURRENT_PROP_D * M_TO_INCH, CURRENT_PROP_D * M_TO_INCH],
        "prop_P [in]": [CURRENT_PROP_P * M_TO_INCH, CURRENT_PROP_P * M_TO_INCH],
        "hull_u [km/h]": [
            res_baseline_sprint["hull_u_kmh"],
            res_case1_sprint["hull_u_kmh"],
        ],
        "Distância [km]": [
            res_baseline_sprint["distance_km"],
            res_case1_sprint["distance_km"],
        ],
        "batt_z final [%]": [
            res_baseline_sprint["batt_z_final"] * 100,
            res_case1_sprint["batt_z_final"] * 100,
        ],
        "motor_q max [Nm]": [
            res_baseline_sprint["motor_q_max"],
            res_case1_sprint["motor_q_max"],
        ],
        "motor_w [rad/s]": [
            res_baseline_sprint["motor_w_mean"],
            res_case1_sprint["motor_w_mean"],
        ],
        "prop_j [-]": [
            res_baseline_sprint["prop_j_mean"],
            res_case1_sprint["prop_j_mean"],
        ],
        "prop_w [rad/s]": [
            res_baseline_sprint["prop_w_mean"],
            res_case1_sprint["prop_w_mean"],
        ],
    }
)
df_sprint["vs Baseline [%]"] = (
    df_sprint["hull_u [km/h]"] / res_baseline_sprint["hull_u_kmh"] - 1
) * 100
display(df_sprint.round(2))


--- Tabela 1: Performance Sprint ---


,Caso,trans_k,esc_d [%],prop_D [in],prop_P [in],hull_u [km/h],Distância [km],batt_z final [%],motor_q max [Nm],motor_w [rad/s],prop_j [-],prop_w [rad/s],vs Baseline [%]
0,Baseline,0.64,100.0,9.0,10.5,18.00,0.24,96.14,15.32,297.27,0.68,189.17,0.00
1,Caso 1,0.93,100.0,9.0,10.5,20.57,0.24,93.27,28.95,232.52,0.68,215.09,14.29


In [23]:
# Tabela 2: Performance Endurance (média ponderada)
print("\n--- Tabela 2: Performance Endurance (média ponderada) ---")

# Calcular médias ponderadas para baseline e caso 1
baseline_end_batt_z = float(
    np.sum(
        np.array([r["batt_z_final"] for r in baseline_endurance_results]) * PV_G_WEIGHTS
    )
)
baseline_end_motor_q = float(
    np.max([r["motor_q_max"] for r in baseline_endurance_results])
)
baseline_end_motor_w = float(
    np.sum(
        np.array([r["motor_w_mean"] for r in baseline_endurance_results]) * PV_G_WEIGHTS
    )
)
baseline_end_prop_j = float(
    np.sum(
        np.array([r["prop_j_mean"] for r in baseline_endurance_results]) * PV_G_WEIGHTS
    )
)
baseline_end_prop_w = float(
    np.sum(
        np.array([r["prop_w_mean"] for r in baseline_endurance_results]) * PV_G_WEIGHTS
    )
)

case1_end_batt_z = float(
    np.sum(
        np.array([r["batt_z_final"] for r in case1_endurance_results]) * PV_G_WEIGHTS
    )
)
case1_end_motor_q = float(np.max([r["motor_q_max"] for r in case1_endurance_results]))
case1_end_motor_w = float(
    np.sum(
        np.array([r["motor_w_mean"] for r in case1_endurance_results]) * PV_G_WEIGHTS
    )
)
case1_end_prop_j = float(
    np.sum(np.array([r["prop_j_mean"] for r in case1_endurance_results]) * PV_G_WEIGHTS)
)
case1_end_prop_w = float(
    np.sum(np.array([r["prop_w_mean"] for r in case1_endurance_results]) * PV_G_WEIGHTS)
)

df_endurance = pd.DataFrame(
    {
        "Caso": ["Baseline", "Caso 1"],
        "trans_k": [CURRENT_TRANS_K, CASE1_TRANS_K_ENDURANCE],
        "esc_d [%]": [BASELINE_ESC_D_ENDURANCE * 100, CASE1_ESC_D_ENDURANCE * 100],
        "prop_D [in]": [CURRENT_PROP_D * M_TO_INCH, CURRENT_PROP_D * M_TO_INCH],
        "prop_P [in]": [CURRENT_PROP_P * M_TO_INCH, CURRENT_PROP_P * M_TO_INCH],
        "hull_u [km/h]": [
            BASELINE_ENDURANCE_SPEED * MPS_TO_KMH,
            CASE1_ENDURANCE_DISTANCE / ENDURANCE_DURATION * MPS_TO_KMH,
        ],
        "Distância [km]": [
            BASELINE_ENDURANCE_DISTANCE / 1000,
            CASE1_ENDURANCE_DISTANCE / 1000,
        ],
        "batt_z final [%]": [baseline_end_batt_z * 100, case1_end_batt_z * 100],
        "motor_q max [Nm]": [baseline_end_motor_q, case1_end_motor_q],
        "motor_w [rad/s]": [baseline_end_motor_w, case1_end_motor_w],
        "prop_j [-]": [baseline_end_prop_j, case1_end_prop_j],
        "prop_w [rad/s]": [baseline_end_prop_w, case1_end_prop_w],
    }
)
df_endurance["vs Baseline [%]"] = (
    df_endurance["Distância [km]"] / (BASELINE_ENDURANCE_DISTANCE / 1000) - 1
) * 100
display(df_endurance.round(2))


--- Tabela 2: Performance Endurance (média ponderada) ---


,Caso,trans_k,esc_d [%],prop_D [in],prop_P [in],hull_u [km/h],Distância [km],batt_z final [%],motor_q max [Nm],motor_w [rad/s],prop_j [-],prop_w [rad/s],vs Baseline [%]
0,Baseline,0.64,40.01,9.0,10.5,8.49,42.44,63.86,3.74,136.47,0.68,86.85,0.0
1,Caso 1,0.48,52.84,9.0,10.5,8.56,42.78,63.65,2.85,183.42,0.68,87.54,0.8


In [24]:
# Tabela 3: Caso 2 - Fronteira de Pareto com todas as métricas
print("\n--- Tabela 3: Caso 2 - Fronteira de Pareto ---")

# Adicionar comparação com baseline
df_pareto["vs Baseline Sprint [%]"] = (
    df_pareto["sprint_speed [m/s]"] / BASELINE_SPRINT_SPEED - 1
) * 100
df_pareto["vs Baseline Endurance [%]"] = (
    df_pareto["endurance_distance [km]"] / (BASELINE_ENDURANCE_DISTANCE / 1000) - 1
) * 100

display(df_pareto.round(2))


--- Tabela 3: Caso 2 - Fronteira de Pareto ---


,trans_k,esc_d_sprint [%],esc_d_endurance [%],prop_D_sprint [in],prop_PD_sprint,prop_P_sprint [in],prop_D_endurance [in],prop_PD_endurance,prop_P_endurance [in],sprint_speed [m/s],sprint_speed [km/h],endurance_distance [km],vs Baseline Sprint [%],vs Baseline Endurance [%]
0,0.51,100.0,43.97,11.81,1.60,18.89,10.90,0.91,9.90,6.00,21.60,43.93,20.00,3.49
1,0.51,100.0,43.97,11.81,1.60,18.87,10.90,0.91,9.90,6.00,21.60,43.93,20.00,3.49
2,0.51,100.0,43.97,11.81,1.60,18.88,10.90,0.91,9.90,6.00,21.60,43.93,20.00,3.49
3,0.48,100.0,43.97,11.66,1.60,18.62,11.22,0.92,10.37,5.85,21.07,44.06,17.07,3.81
4,0.48,100.0,43.97,11.68,1.58,18.41,11.22,0.92,10.37,5.85,21.07,44.06,17.07,3.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.48,100.0,43.97,11.70,1.60,18.70,11.22,0.92,10.37,5.85,21.07,44.06,17.07,3.81
96,0.48,100.0,43.97,11.80,1.54,18.12,11.22,0.92,10.37,5.85,21.07,44.06,17.07,3.81
97,0.48,100.0,43.97,11.61,1.58,18.39,11.22,0.92,10.37,5.85,21.07,44.06,17.07,3.81
98,0.48,100.0,43.97,11.70,1.56,18.29,11.22,0.92,10.37,5.85,21.07,44.06,17.07,3.81


## Resumo Final

In [25]:
print("\n" + "=" * 80)
print("RESUMO FINAL")
print("=" * 80)

print(f"""
================================================================================
BASELINE (Configuração Atual)
================================================================================
  trans_k: {CURRENT_TRANS_K:.4f}
  Hélice: D={CURRENT_PROP_D * M_TO_INCH:.2f}" P={CURRENT_PROP_P * M_TO_INCH:.2f}"

  --- Sprint ---
    esc_d: {SPRINT_ESC_D * 100:.0f}%
    hull_u: {BASELINE_SPRINT_SPEED * MPS_TO_KMH:.2f} km/h
    Distância: {res_baseline_sprint["distance_km"]:.3f} km
    batt_z final: {res_baseline_sprint["batt_z_final"] * 100:.1f}%
    motor_q max: {res_baseline_sprint["motor_q_max"]:.2f} Nm
    motor_w: {res_baseline_sprint["motor_w_mean"]:.1f} rad/s
    prop_j: {res_baseline_sprint["prop_j_mean"]:.3f}
    prop_w: {res_baseline_sprint["prop_w_mean"]:.1f} rad/s

  --- Endurance (média ponderada) ---
    esc_d: {BASELINE_ESC_D_ENDURANCE * 100:.1f}%
    hull_u: {BASELINE_ENDURANCE_SPEED * MPS_TO_KMH:.2f} km/h
    Distância: {BASELINE_ENDURANCE_DISTANCE / 1000:.2f} km
    batt_z final: {baseline_end_batt_z * 100:.1f}%
    motor_q max: {baseline_end_motor_q:.2f} Nm
    motor_w: {baseline_end_motor_w:.1f} rad/s
    prop_j: {baseline_end_prop_j:.3f}
    prop_w: {baseline_end_prop_w:.1f} rad/s

================================================================================
CASO 1 (Otimização de trans_k, hélice fixo)
================================================================================
  --- Sprint ---
    trans_k: {CASE1_TRANS_K_SPRINT:.4f}
    esc_d: {CASE1_ESC_D_SPRINT * 100:.0f}%
    prop_D: {CURRENT_PROP_D * M_TO_INCH:.2f}"
    prop_P: {CURRENT_PROP_P * M_TO_INCH:.2f}"
    hull_u: {CASE1_SPRINT_SPEED * MPS_TO_KMH:.2f} km/h ({(CASE1_SPRINT_SPEED / BASELINE_SPRINT_SPEED - 1) * 100:+.1f}% vs baseline)
    Distância: {res_case1_sprint["distance_km"]:.3f} km
    batt_z final: {res_case1_sprint["batt_z_final"] * 100:.1f}%
    motor_q max: {res_case1_sprint["motor_q_max"]:.2f} Nm
    motor_w: {res_case1_sprint["motor_w_mean"]:.1f} rad/s
    prop_j: {res_case1_sprint["prop_j_mean"]:.3f}
    prop_w: {res_case1_sprint["prop_w_mean"]:.1f} rad/s

  --- Endurance (média ponderada) ---
    trans_k: {CASE1_TRANS_K_ENDURANCE:.4f}
    esc_d: {CASE1_ESC_D_ENDURANCE * 100:.1f}%
    prop_D: {CURRENT_PROP_D * M_TO_INCH:.2f}"
    prop_P: {CURRENT_PROP_P * M_TO_INCH:.2f}"
    hull_u: {CASE1_ENDURANCE_DISTANCE / ENDURANCE_DURATION * MPS_TO_KMH:.2f} km/h
    Distância: {CASE1_ENDURANCE_DISTANCE / 1000:.2f} km ({(CASE1_ENDURANCE_DISTANCE / BASELINE_ENDURANCE_DISTANCE - 1) * 100:+.1f}% vs baseline)
    batt_z final: {case1_end_batt_z * 100:.1f}%
    motor_q max: {case1_end_motor_q:.2f} Nm
    motor_w: {case1_end_motor_w:.1f} rad/s
    prop_j: {case1_end_prop_j:.3f}
    prop_w: {case1_end_prop_w:.1f} rad/s
""")


RESUMO FINAL

BASELINE (Configuração Atual)
  trans_k: 0.6364
  Hélice: D=9.00" P=10.50"

  --- Sprint ---
    esc_d: 100%
    hull_u: 18.00 km/h
    Distância: 0.240 km
    batt_z final: 96.1%
    motor_q max: 15.32 Nm
    motor_w: 297.3 rad/s
    prop_j: 0.677
    prop_w: 189.2 rad/s

  --- Endurance (média ponderada) ---
    esc_d: 40.0%
    hull_u: 8.49 km/h
    Distância: 42.44 km
    batt_z final: 63.9%
    motor_q max: 3.74 Nm
    motor_w: 136.5 rad/s
    prop_j: 0.677
    prop_w: 86.8 rad/s

CASO 1 (Otimização de trans_k, hélice fixo)
  --- Sprint ---
    trans_k: 0.9250
    esc_d: 100%
    prop_D: 9.00"
    prop_P: 10.50"
    hull_u: 20.57 km/h (+14.3% vs baseline)
    Distância: 0.240 km
    batt_z final: 93.3%
    motor_q max: 28.95 Nm
    motor_w: 232.5 rad/s
    prop_j: 0.677
    prop_w: 215.1 rad/s

  --- Endurance (média ponderada) ---
    trans_k: 0.4773
    esc_d: 52.8%
    prop_D: 9.00"
    prop_P: 10.50"
    hull_u: 8.56 km/h
    Distância: 42.78 km (+0.8% vs baseli

In [26]:
# Resumo detalhado do Caso 2
print(f"""
================================================================================
CASO 2 (Multi-objetivo com dois hélices - NSGA-II)
================================================================================
  Número de soluções na fronteira de Pareto: {len(df_pareto)}
  trans_k único para ambas as provas (otimizado)
  esc_d sprint fixo em {SPRINT_ESC_D * 100:.0f}%
  esc_d endurance otimizado
  Dois hélices diferentes: um para sprint, outro para endurance
""")

# Identificar soluções extremas e balanceada
idx_best_sprint = df_pareto["sprint_speed [km/h]"].idxmax()
idx_best_endurance = df_pareto["endurance_distance [km]"].idxmax()

# Solução balanceada: mais próxima da diagonal normalizada
sprint_norm = (
    df_pareto["sprint_speed [km/h]"] - df_pareto["sprint_speed [km/h]"].min()
) / (
    df_pareto["sprint_speed [km/h]"].max()
    - df_pareto["sprint_speed [km/h]"].min()
    + 1e-9
)
endurance_norm = (
    df_pareto["endurance_distance [km]"] - df_pareto["endurance_distance [km]"].min()
) / (
    df_pareto["endurance_distance [km]"].max()
    - df_pareto["endurance_distance [km]"].min()
    + 1e-9
)
df_pareto["balance_score"] = sprint_norm + endurance_norm
idx_balanced = df_pareto["balance_score"].idxmax()

for idx, label in [
    (idx_best_sprint, "MELHOR SPRINT"),
    (idx_best_endurance, "MELHOR ENDURANCE"),
    (idx_balanced, "BALANCEADA"),
]:
    row = df_pareto.loc[idx]
    print(f"""
  --- Solução {label} ---
    trans_k: {row["trans_k"]:.4f}
    
    Sprint:
      esc_d: {row["esc_d_sprint [%]"]:.0f}%
      prop_D: {row["prop_D_sprint [in]"]:.2f}"
      prop_P: {row["prop_P_sprint [in]"]:.2f}"
      hull_u: {row["sprint_speed [km/h]"]:.2f} km/h ({row["vs Baseline Sprint [%]"]:+.1f}% vs baseline)
    
    Endurance:
      esc_d: {row["esc_d_endurance [%]"]:.1f}%
      prop_D: {row["prop_D_endurance [in]"]:.2f}"
      prop_P: {row["prop_P_endurance [in]"]:.2f}"
      Distância: {row["endurance_distance [km]"]:.2f} km ({row["vs Baseline Endurance [%]"]:+.1f}% vs baseline)
""")

# Limpar coluna temporária
df_pareto.drop(columns=["balance_score"], inplace=True, errors="ignore")


CASO 2 (Multi-objetivo com dois hélices - NSGA-II)
  Número de soluções na fronteira de Pareto: 100
  trans_k único para ambas as provas (otimizado)
  esc_d sprint fixo em 100%
  esc_d endurance otimizado
  Dois hélices diferentes: um para sprint, outro para endurance


  --- Solução MELHOR SPRINT ---
    trans_k: 0.5066
    
    Sprint:
      esc_d: 100%
      prop_D: 11.81"
      prop_P: 18.89"
      hull_u: 21.60 km/h (+20.0% vs baseline)
    
    Endurance:
      esc_d: 44.0%
      prop_D: 10.90"
      prop_P: 9.90"
      Distância: 43.93 km (+3.5% vs baseline)


  --- Solução MELHOR ENDURANCE ---
    trans_k: 0.4800
    
    Sprint:
      esc_d: 100%
      prop_D: 11.66"
      prop_P: 18.62"
      hull_u: 21.07 km/h (+17.1% vs baseline)
    
    Endurance:
      esc_d: 44.0%
      prop_D: 11.22"
      prop_P: 10.37"
      Distância: 44.06 km (+3.8% vs baseline)


  --- Solução BALANCEADA ---
    trans_k: 0.5066
    
    Sprint:
      esc_d: 100%
      prop_D: 11.81"
      prop_P:

In [27]:
df_endurance.to_csv("data/optimization_unified_endurance.csv")
df_sprint.to_csv("data/optimization_unified_sprint.csv")
df_pareto.to_csv("data/optimization_unified_pareto.csv")